In [1]:
import pandas as pd
import psycopg2

CONNECT_DB = "host=localhost port=5432 dbname=university_db user=dataengineer password=secret"

In [2]:
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor

# Connection string
CONNECT_DB = "host=localhost port=5432 dbname=university_db user=dataengineer password=secret"

def run_query(query):
    """
    Execute a SQL query and return the result as a DataFrame.
    """
    try:
        # Establish connection
        conn = psycopg2.connect(CONNECT_DB)
        
        # Execute query and fetch data
        with conn.cursor(cursor_factory=RealDictCursor) as cursor:
            cursor.execute(query)
            data = cursor.fetchall()
        
        # Convert fetched data to DataFrame
        df = pd.DataFrame(data)
        return df
    
    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        # Close connection
        if conn:
            conn.close()


In [76]:
query = '''
SELECT schedule_id, semester, COUNT(student_id)
FROM wh__enrollments
GROUP BY schedule_id, semester
ORDER BY semester, schedule_id

'''

result_df = run_query(query)
result_df

,schedule_id,semester,count
0,1,1,2
1,2,1,3
2,3,1,2
3,4,1,2
4,5,2,1
5,7,2,2
6,8,2,1
7,10,2,2


In [16]:
query = '''
SELECT attend_dt, schedule_id, COUNT(student_id)
FROM wh__attendances
GROUP BY schedule_id, attend_dt
'''

result_df = run_query(query)
result_df

,attend_dt,schedule_id,count
0,2019-11-01,1,2
1,2020-02-12,7,2
2,2019-11-19,2,3
3,2020-04-03,10,2
4,2019-10-21,1,2
...,...,...,...
209,2020-03-26,10,2
210,2019-10-25,1,1
211,2019-10-07,1,2
212,2020-03-02,7,2


In [77]:
query = '''
SELECT course_id, schedule_date, week_number,
        CASE WHEN schedule_date < '2019-12-31' THEN 1
             WHEN schedule_date > '2020-01-01' THEN 2
             END AS semester
    FROM wh__schedules
'''

result_df = run_query(query)
result_df

,course_id,schedule_date,week_number,semester
0,1,2019-09-09,1,1
1,1,2019-09-11,1,1
2,1,2019-09-13,1,1
3,1,2019-09-16,2,1
4,1,2019-09-18,2,1
...,...,...,...,...
255,3,2020-04-10,11,2
256,3,2020-04-16,12,2
257,3,2020-04-17,12,2
258,3,2020-04-23,13,2


In [81]:
query = '''
WITH enrollment_num AS (
    SELECT schedule_id, semester, COUNT(student_id) AS student_enr
    FROM wh__enrollments
    GROUP BY schedule_id, semester
    ORDER BY semester, schedule_id
),
schedule_sum AS (
    SELECT course_id, schedule_date, week_number,
        CASE WHEN schedule_date < '2019-12-31' THEN 1
             WHEN schedule_date > '2020-01-01' THEN 2
             END AS semester
    FROM wh__schedules
),
attendance_sum AS (
    SELECT attend_dt, schedule_id, COUNT(student_id) AS student_atd
    FROM wh__attendances
    GROUP BY schedule_id, attend_dt
)
SELECT
    s.course_id,
    s.schedule_date,
    s.week_number,
    s.semester,
    COALESCE(student_atd, 0) AS student_attend,
    COALESCE(e.student_enr, 0) AS student_enrolled
FROM schedule_sum AS s
    LEFT JOIN attendance_sum AS a
        ON s.course_id = a.schedule_id
        AND s.schedule_date = a.attend_dt
    LEFT JOIN enrollment_num AS e
        ON s.course_id = e.schedule_id
        AND s.semester = e.semester
WHERE course_id = 1
AND week_number = 1
'''

result_df = run_query(query)
result_df.head(20)

,course_id,schedule_date,week_number,semester,student_attend,student_enrolled
0,1,2019-09-09,1,1,2,2
1,1,2019-09-11,1,1,2,2
2,1,2019-09-13,1,1,1,2
3,1,2020-01-27,1,2,0,0
4,1,2020-01-29,1,2,0,0
5,1,2020-01-31,1,2,0,0


In [104]:
query = '''
WITH enrollment_num AS (
    SELECT schedule_id, semester, COUNT(student_id) AS student_enr
    FROM wh__enrollments
    GROUP BY schedule_id, semester
    ORDER BY semester, schedule_id
),
schedule_sum AS (
    SELECT course_id, schedule_date, week_number,
        CASE WHEN schedule_date < '2019-12-31' THEN 1
             WHEN schedule_date > '2020-01-01' THEN 2
             END AS semester
    FROM wh__schedules
),
attendance_sum AS (
    SELECT attend_dt, schedule_id, COUNT(student_id) AS student_atd
    FROM wh__attendances
    GROUP BY schedule_id, attend_dt
),
attendance_pct AS (
    SELECT
        s.course_id,
        s.schedule_date,
        s.week_number,
        s.semester,
        COALESCE(student_atd, 0) AS student_attend,
        COALESCE(e.student_enr, 0) AS student_enrolled
    FROM schedule_sum AS s
        LEFT JOIN attendance_sum AS a
            ON s.course_id = a.schedule_id
            AND s.schedule_date = a.attend_dt
        LEFT JOIN enrollment_num AS e
            ON s.course_id = e.schedule_id
            AND s.semester = e.semester
)
SELECT
    --p.course_id,
    c.name AS course_name,
    p.semester,
    p.week_number,
    --SUM(p.student_attend) AS total_attend,
    --SUM(p.student_enrolled) AS total_student,
    ROUND((SUM(p.student_attend) / SUM(p.student_enrolled) * 100), 2) AS attendance_percentage
FROM attendance_pct AS p
LEFT JOIN wh__courses AS c
    ON p.course_id = c.id
GROUP BY p.semester, c.name, p.course_id, p.week_number
HAVING SUM(p.student_enrolled) != 0
ORDER BY p.semester, p.course_id, p.week_number
'''

result_df = run_query(query)
result_df.to_csv('1.csv')

In [101]:
query = '''
WITH schedule_sum AS (
    SELECT
        course_id,
        schedule_date,
        week_number,
        CASE
            WHEN schedule_date < '2019-12-31' THEN 1
            WHEN schedule_date > '2020-01-01' THEN 2
        END AS semester
    FROM wh__schedules
),
attendance_sum AS (
    SELECT
        schedule_id,
        attend_dt,
        COUNT(student_id) AS student_atd
    FROM wh__attendances
    GROUP BY schedule_id, attend_dt
),
enrollment_num AS (
    SELECT
        schedule_id,
        COUNT(student_id) AS student_enr
    FROM wh__enrollments
    GROUP BY schedule_id
),
attendance_pct AS (
    SELECT
        s.course_id,
        s.schedule_date,
        s.week_number,
        s.semester,
        COALESCE(a.student_atd, 0) AS student_attend,
        COALESCE(e.student_enr, 0) AS student_enrolled
    FROM schedule_sum AS s
    LEFT JOIN attendance_sum AS a
        ON s.course_id = a.schedule_id
        AND s.schedule_date = a.attend_dt
    LEFT JOIN enrollment_num AS e
        ON s.course_id = e.schedule_id
)
SELECT
    c.name AS course_name,
    p.semester,
    p.week_number,
    ROUND((SUM(p.student_attend) / NULLIF(SUM(p.student_enrolled), 0) * 100), 2) AS attendance_percentage
FROM attendance_pct AS p
LEFT JOIN wh__courses AS c
    ON p.course_id = c.id
GROUP BY p.semester, c.name, p.course_id, p.week_number
HAVING SUM(p.student_enrolled) > 0
ORDER BY p.semester, p.course_id, p.week_number
'''

result_df = run_query(query)
result_df.to_csv('2.csv')